# 📊 Preparação de Dados para Fine-Tuning

Transforma seus **103 markdowns** em dataset para fine-tuning.

## 🔧 Setup

In [ ]:
import sys
sys.path.append('../')

from src.finetuning.data_prep import DatasetPreparator, format_instruction_dataset
from pathlib import Path
import re
import shutil

## 📁 Configuração

In [ ]:
DATA_DIR = Path("../data/finetuning")
DATA_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DATA_DIR = DATA_DIR / "processed"
PROCESSED_DATA_DIR.mkdir(exist_ok=True)
MARKDOWN_DIR = Path("../data/input")

print(f"📁 Seus markdowns: {MARKDOWN_DIR}")
print(f"📊 Total: {len(list(MARKDOWN_DIR.glob('*.md')))} arquivos")

📁 Seus markdowns: ../data/input
📊 Total: 107 arquivos


## 🎯 Extrair Q&A dos Markdowns

In [ ]:
def extract_qa_from_markdowns(markdown_dir):
    """Extrai Q&A dos markdowns."""
    qa_pairs = []
    
    for md_file in Path(markdown_dir).glob("*.md"):
        try:
            with open(md_file, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # Pegar título (linha com #)
            lines = content.split('\n')
            title = None
            for line in lines:
                if line.startswith('# '):
                    title = line.replace('# ', '').strip()
                    break
            
            if not title:
                title = md_file.stem
            
            # Remover metadados do final (---)
            if '---' in content:
                content = content.split('---')[0]
            
            # Limpar markdown (negrito, itálico, links)
            content_clean = content
            content_clean = re.sub(r'\*\*(.+?)\*\*', r'\1', content_clean)
            content_clean = re.sub(r'\*(.+?)\*', r'\1', content_clean)
            content_clean = re.sub(r'\[(.+?)\]\(.+?\)', r'\1', content_clean)
            
            # Encontrar seções com ##
            sections = []
            current_section = None
            current_content = []
            
            for line in content_clean.split('\n'):
                if line.startswith('## '):
                    if current_section:
                        sections.append((current_section, '\n'.join(current_content)))
                    current_section = line.replace('## ', '').strip()
                    current_content = []
                elif current_section:
                    current_content.append(line)
            
            if current_section:
                sections.append((current_section, '\n'.join(current_content)))
            
            # Se tem seções, criar Q&A de cada seção
            if sections:
                for section_title, section_content in sections:
                    clean_content = section_content.strip()
                    if len(clean_content) > 100:
                        # Limitar tamanho
                        answer = clean_content[:1000] if len(clean_content) > 1000 else clean_content
                        qa_pairs.append({
                            "instruction": section_title + '?',
                            "output": answer.strip()
                        })
            
            # Sempre criar uma pergunta geral sobre o documento
            paragraphs = [p.strip() for p in content_clean.split('\n\n') if len(p.strip()) > 100]
            if paragraphs:
                # Pegar primeiros 2-3 parágrafos
                intro = '\n\n'.join(paragraphs[:2])
                if len(intro) > 100:
                    answer = intro[:1200] if len(intro) > 1200 else intro
                    qa_pairs.append({
                        "instruction": title + '?',
                        "output": answer.strip()
                    })
        
        except Exception as e:
            print(f"Erro ao processar {md_file.name}: {e}")
    
    return qa_pairs

print("🔄 Processando seus markdowns...")
qa_pairs = extract_qa_from_markdowns(MARKDOWN_DIR)
print(f"\n✅ Extraídos {len(qa_pairs)} pares Q&A!")

if qa_pairs:
    print(f"\n📝 Primeiro exemplo:")
    print(f"Pergunta: {qa_pairs[0]['instruction']}")
    print(f"Resposta: {qa_pairs[0]['output'][:200]}...")
else:
    print("⚠️ Nenhum Q&A extraído. Verifique os arquivos markdown.")

🔄 Processando seus markdowns...

✅ Extraídos 107 pares Q&A!

📝 Primeiro exemplo:
Pergunta: 5 Tipos de anúncios no Mercado Livre: Guia Completo!?
Resposta: Se você vende no Mercado Livre, é importante conhecer os tipos de anúncios disponíveis e suas características. Cada tipo possui vantagens e desvantagens, por isso, é importante saber qual opção vale m...


## 🔄 Formatar Dataset

## ⚠️ DIAGNÓSTICO: Tamanho do Dataset

**IMPORTANTE**: Você vai precisar de pelo menos **500-1000 exemplos** para fine-tuning efetivo.

Com 107 markdowns, você vai gerar aproximadamente **~100-200 pares Q&A**.

Isso é **MUITO POUCO** e pode causar:
- ❌ Catastrophic forgetting (modelo esquece conhecimento geral)
- ❌ Overfitting extremo (decora os exemplos)
- ❌ Respostas sem sentido durante inferência

**Recomendação**: Use RAG sem fine-tuning. Veja `docs/FINE_TUNING_VS_RAG.md`


In [ ]:
if not qa_pairs:
    raise ValueError("❌ Nenhum dado foi extraído! Verifique os markdowns.")

dataset = format_instruction_dataset(qa_pairs, template="alpaca")
print(f"✅ Dataset formatado com {len(dataset)} exemplos!")
print(f"\n📝 Exemplo formatado:")
print("=" * 80)
print(dataset[0]['text'][:500] + "...")
print("=" * 80)

✅ Dataset formatado com 107 exemplos!

📝 Exemplo formatado:
### Instruction:
5 Tipos de anúncios no Mercado Livre: Guia Completo!?

### Response:
Se você vende no Mercado Livre, é importante conhecer os tipos de anúncios disponíveis e suas características. Cada tipo possui vantagens e desvantagens, por isso, é importante saber qual opção vale mais a pena para as suas vendas. Você sabia que existem 5 tipos de anúncios no Mercado Livre? No artigo de hoje, nosso CEO e consultor certificado Joel Jonathan Cunha vai lhe apresentar em detalhes quais são e como ...


## 🔀 Dividir Treino/Validação

In [ ]:
preparator = DatasetPreparator(template="alpaca")
split_dataset = preparator.create_train_test_split(dataset, test_size=0.1, seed=42)
print(f"📊 Dataset dividido:")
print(f"   Treino: {len(split_dataset['train'])} exemplos")
print(f"   Validação: {len(split_dataset['test'])} exemplos")

📊 Dataset dividido:
   Treino: 96 exemplos
   Validação: 11 exemplos


## ✅ Validar Qualidade

In [ ]:
texts = [item['text'] for item in split_dataset['train']]
lengths = [len(text) for text in texts]

print("📊 Estatísticas:")
print(f"   Total: {len(texts)} exemplos")
print(f"   Tamanho médio: {sum(lengths) / len(lengths):.0f} caracteres")
print(f"   Mínimo: {min(lengths)} | Máximo: {max(lengths)}")

print(f"\n📝 3 exemplos do treino:")
for i in range(min(3, len(split_dataset['train']))):
    print(f"\n{i+1}. {split_dataset['train'][i]['text'][:250]}...\n{'-'*80}")

📊 Estatísticas:
   Total: 96 exemplos
   Tamanho médio: 1302 caracteres
   Mínimo: 1271 | Máximo: 1338

📝 3 exemplos do treino:

1. ### Instruction:
Anúncio patrocinado no Mercado Livre: tudo que você precisa saber para aumentar as suas vendas.?

### Response:
Para aumentar a visibilidade e a exposição dos seus produtos no Mercado Livre você precisará adotar estratégias que não d...
--------------------------------------------------------------------------------

2. ### Instruction:
Ficha Técnica nos Anúncios do Mercado Livre: não deixe em branco!?

### Response:
Ao contrário do que alguns vendedores podem pensar, aficha técnica nos anúncios do Mercado Livreé muito importante para aumentar as vendas. Além de aju...
--------------------------------------------------------------------------------

3. ### Instruction:
Ruptura de estoque nos anúncios do Mercado Livre: não deixe isso acontecer!?

### Response:
A gestão eficiente do estoque é um dos pilares para o sucesso das vendas on-line.

## 💾 Salvar

In [ ]:
# 🎯 DIAGNÓSTICO FINAL
train_size = len(split_dataset['train'])

print("="*80)
print("📊 DIAGNÓSTICO FINAL DO DATASET")
print("="*80)
print(f"\n📈 Tamanho atual:")
print(f"   • Treino: {train_size} exemplos")
print(f"   • Validação: {len(split_dataset['test'])} exemplos")
print(f"   • Total: {train_size + len(split_dataset['test'])} exemplos")

print(f"\n🎯 Recomendações para fine-tuning:")
print(f"   • Mínimo viável: 500 exemplos")
print(f"   • Recomendado: 1000-2000 exemplos")
print(f"   • Ideal: 5000+ exemplos")

if train_size < 500:
    print(f"\n⚠️  STATUS: DATASET MUITO PEQUENO")
    print(f"   Você tem apenas {(train_size/500)*100:.1f}% do mínimo recomendado")
    print(f"\n❌ Fazer fine-tuning agora vai resultar em:")
    print(f"   • Respostas sem sentido")
    print(f"   • Alucinações (tópicos aleatórios)")
    print(f"   • Perda de conhecimento geral")
    print(f"   • Overfitting extremo")
    
    print(f"\n✅ RECOMENDAÇÕES:")
    print(f"\n   1️⃣  USE RAG SEM FINE-TUNING (MELHOR OPÇÃO)")
    print(f"       → Seu RAG já funciona bem")
    print(f"       → {train_size + len(split_dataset['test'])} markdowns são suficientes para RAG")
    print(f"       → Sem risco de catastrophic forgetting")
    
    print(f"\n   2️⃣  Aumente o dataset antes de fazer fine-tuning:")
    print(f"       → Data augmentation com GPT-4")
    print(f"       → Colete logs de perguntas reais")
    print(f"       → Extraia mais Q&A dos markdowns")
    print(f"       → Crie variações das perguntas")
    
    print(f"\n   3️⃣  Leia a documentação:")
    print(f"       → docs/FINE_TUNING_VS_RAG.md")
    print(f"       → Explica quando usar cada abordagem")
    
elif train_size < 1000:
    print(f"\n⚠️  STATUS: DATASET PEQUENO")
    print(f"   Pode funcionar, mas ainda há risco de overfitting")
    print(f"   Considere aumentar para 1000+ exemplos")
else:
    print(f"\n✅ STATUS: DATASET ADEQUADO")
    print(f"   Tamanho suficiente para fine-tuning")

print("\n" + "="*80)
print(f"💡 LEMBRE-SE: Para seu caso específico (Retail Media),")
print(f"   RAG provavelmente vai dar melhores resultados que fine-tuning!")
print("="*80)


In [ ]:
output_path = PROCESSED_DATA_DIR / "training_dataset"
preparator.save_dataset(split_dataset, output_path)
print(f"✅ Salvo em: {output_path}")

Saving the dataset (0/1 shards):   0%|          | 0/96 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11 [00:00<?, ? examples/s]

✅ Dataset salvo em: ../data/finetuning/processed/training_dataset
✅ Salvo em: ../data/finetuning/processed/training_dataset


## 📦 Comprimir

In [ ]:
output_zip = PROCESSED_DATA_DIR / "training_dataset.zip"
shutil.make_archive(str(output_zip.with_suffix('')), 'zip', output_path)
print(f"✅ ZIP criado: {output_zip}")
print(f"📦 Tamanho: {output_zip.stat().st_size / 1024 / 1024:.2f} MB")
print(f"\n🚀 Próximo passo:")
print(f"   1. Faça upload para Google Drive")
print(f"   2. Use: fine_tuning_qlora_colab.ipynb")

✅ ZIP criado: ../data/finetuning/processed/training_dataset.zip
📦 Tamanho: 0.04 MB

🚀 Próximo passo:
   1. Faça upload para Google Drive
   2. Use: fine_tuning_qlora_colab.ipynb
